In [33]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import pyjokes
from keras.models import load_model
import json
import random
import tensorflow as tf   
import pandas as pd
tf.get_logger().setLevel('ERROR')
import convert

In [34]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [35]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [36]:
def predict_class(sentence, model , words , classes):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.1
    results = [[i,r] for i,r in enumerate(res) if r>=ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
#     print(return_list)
    return return_list

In [48]:
def getResponse(intent , qtype):
    lisp = []
    if qtype == 'type1':
        data_file = open('database/intents1.json').read()
        lisp = json.loads(data_file)["type1"]
    elif qtype == 'type2':
        data_file = open('database/intents2.json').read()
        lisp = json.loads(data_file)["type2"]
    elif qtype == 'type3':
        data_file = open('database/intents3.json').read()
        lisp = json.loads(data_file)["type3"]
        
    for i in lisp:
        if(i['tag']== intent):
            result = random.choice(i['responses'])
            break
    return result

In [39]:
def call_model(sentence , qtype):
    no = qtype
    model = load_model('models/model' + no + '.h5')
    words = pickle.load(open('type' + no + '/words.pkl','rb'))
    classes = pickle.load(open('type' + no + '/classes.pkl','rb'))
    found_intent = predict_class(sentence , model , words , classes)[0]['intent']
#     print("Intent is " , found_intent)
    return found_intent

In [40]:
def main(question):
#     print("Finding the type")
    model = load_model('models/model.h5')
    words = pickle.load(open('main_model/words.pkl','rb'))
    classes = pickle.load(open('main_model/classes.pkl','rb'))
    qtype = predict_class(question , model , words , classes)[0]['intent']
#     print("\nQuestion type is" , qtype , end='\n\n')
    if qtype == 'type1':
        it = call_model(question , '1')
    elif qtype == 'type2':
        it = call_model(question , '2')
    elif qtype == 'type3':
        it = call_model(question , '3')
    return (qtype, it)

In [41]:
main("how are you")

('type1', 'greeting')

In [60]:
def solve(lisp):
    data = {}
    data['question'] = lisp
    data['intent'] = []
    data['types'] = []
    for question in lisp:
        print(question)
        a = main(question)
        data['intent'].append(a[1])
        data['types'].append(a[0])
        if a[1] == 'suggest_movie':
            print('Response : Here is a movie suggestion' , convert.give_suggestion('movies'))
        elif a[1] == 'suggest_song':
            print('Response : Here is a song suggestion' , convert.give_suggestion('songs'))
        elif a[1] == 'suggest_book':
            print('Response : Here is a book suggestion' , convert.give_suggestion('books'))
        elif a[1] == 'tell_joke':
            print("Response : Here is a joke" , pyjokes.get_joke('en'))
        elif a[1] == 'get_info':
            print("Response :" , convert.fetch_wikipedia(question))
        else:
            typeq = a[0]
            intentq = a[1]
            print("Response :" , getResponse(intentq , typeq))
        print()
    data = pd.DataFrame(data)
    print(data)

In [56]:
lisp = ["campus of iit mandi", "joining programming club ","how are you","why is competetive programming hard", "who are you", "who created you"]
# lisp = ["suggest me a song" , "suggest me a movie" , "tell me a joke" , "hello how are you"]
solve(lisp)

campus of iit mandi
Response : Campus of IIT Mandi is very beautiful having scenic beauty as cherry on the cake,also it is a 2nd gen IIT so it is equipped with all the modern facilities.

joining programming club 
Response : KamandPrompt,the programming club of IIT Mandi is open for all students willing to learn coding and participate actively.
You can join it by contacting the coordinators or the club members.
There is no criteria for joining and no prerequisites

how are you
Response : Hello

why is competetive programming hard
Response : KamandPrompt,the programming club of IIT Mandi is open for all students willing to learn coding and participate actively.
You can join it by contacting the coordinators or the club members.
There is no criteria for joining and no prerequisites

who are you
Response : Team Prometheus from IIT Mandi :)

who created you
Response : I was made by Team Prometheus which is also my name :)

                              question                         inte

In [180]:
print(convert.give_suggestion('books'))

{'authors': ['Jaideva C. Goswami', ' Andrew K. Chan'], 'image': 'http://libgen.lc//covers/9000/0649d4c0dc548a91adbe1199865dbf9f-d.jpg', 'download': 'http://libgen.lc/ads.php?md5=0649d4c0dc548a91adbe1199865dbf9f', 'publisher': 'Wiley', 'year': '1999', 'pages': '319', 'language': 'English', 'size': '4 MB', 'name': 'Fundamentals of wavelets: theory, algorithms, and applications '}


In [57]:

print(wikipedia.summary("tell me about wikipedia"))

Wait Wait... Don't Tell Me! is an hour-long weekly news radio panel show produced by WBEZ and National Public Radio (NPR) in Chicago, Illinois. On the program, panelists and contestants are quizzed in humorous ways about that week's news. It is distributed by NPR in the United States, internationally on NPR Worldwide and on the Internet via podcast, and typically broadcast on weekends by member stations. The show averages about 6 million weekly listeners on air and via podcast.


["Complaining about the lack of smoking shelters, the nicotine addicted Python programmers said there ought to be 'spaces for tabs'.",
 'Ubuntu users are apt to get this joke.',
 'Obfuscated Reality Mappers (ORMs) can be useful database tools.',
 'Asked to explain Unicode during an interview, Geoff went into detail about his final year university project. He was not hired.',
 "Triumphantly, Beth removed Python 2.7 from her server in 2030. 'Finally!' she said with glee, only to see the announcement for Python 4.4.",
 "An SQL query goes into a bar, walks up to two tables and asks, 'Can I join you?'",
 'When your hammer is C++, everything begins to look like a thumb.',
 'If you put a million monkeys at a million keyboards, one of them will eventually write a Java program. The rest of them will write Perl.',
 'To understand recursion you must first understand recursion.',
 "I suggested holding a 'Python Object Oriented Programming Seminar', but the acronym was unpopular.",
 "'Knock, knock.

In [59]:


sentence = "tell me about Elon Musk"
tagged_sent = pos_tag(sentence.split())
pn = [word for word,pos in tagged_sent if pos == 'NNP']
final = "".join(pn)
print(final)
print(wikipedia.summary(final))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pranshu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


ElonMusk
Elon Reeve Musk  ( EE-lon; born June 28, 1971) is a business magnate, industrial designer and engineer.  He is the founder, CEO, CTO and chief designer of SpaceX; early investor, CEO and product architect of Tesla, Inc.; founder of The Boring Company; co-founder of Neuralink; and co-founder and initial co-chairman of OpenAI. He was elected a Fellow of the Royal Society (FRS) in 2018. Also that year, he was ranked 25th on the Forbes list of The World's Most Powerful People, and was ranked joint-first on the Forbes list of the Most Innovative Leaders of 2019. A centi-billionaire, Musk became the richest person in the world, for a week, in January 2021.Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada when he was 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economic